In [2]:
    # 이미지 링크 가져오기
#     img_src = item.find_element_by_class_name('product_img').get_attribute('src')
    
    # 배경 색상 가져오기
#     bg_color = item.find_element_by_class_name('product').get_attribute('style')


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm_notebook

# 크롤링

In [2]:
# 크롬 드라이버매니저 설치
driver = webdriver.Chrome(ChromeDriverManager().install())

# 크림 url 정보 입력
url = "https://kream.co.kr/search?sort=popular&per_page=20"

# 크림 url 가져오기
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\teysi\.wdm\drivers\chromedriver\win32\98.0.4758.102\chromedriver.exe] found in cache


In [3]:
# 모든 페이지 불러오기 위해 스크롤 내리기
for i in range(10):
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
print("페이지 스크롤 완료")

In [4]:
product_infos = []

# 페이지 내의 상품 리스트를 가져옴
items = driver.find_element_by_class_name('search_result_list')
item_list = items.find_elements_by_class_name('search_result_item')

for item in item_list:
    # 제목, 가격 등 기본 정보 가져오기
    info = item.find_element_by_class_name('item_inner')
    text = info.text.split("\n")
    
    info2 = item.find_element_by_class_name('interest_figure')
    interest = info2.text.split("\n")
    
    #인덱싱으로 브랜드, 이름, 리셀가격, 빠른배송 여부, 좋아요, 위시 긁어오기
    if len(text) == 6:
        brand = text[0]
        en_name = text[1]
        kor_name = text[2]
        re_price = text[3][:-1]
        speed_deli = 1
    else:
        brand = text[0]
        en_name = text[1]
        kor_name = text[2]
        re_price = text[3][:-1]
        speed_deli = 0
    wish = interest[0]
    review = interest[1]
    
    # 상품 세부 페이지 link 가져오기
    link = info.get_attribute('href')

    # 저장
    item_dict = {
        "brand": brand,
        "en_name": en_name,
        "kor_name": kor_name,
        "re_price": re_price,
        "speed_deli": speed_deli,
        "wish": wish,
        "review": review,
        'link': link
    }

    product_infos.append(item_dict)

print("크롤링 완료")
df = pd.DataFrame(product_infos)
df.to_csv("process_1.csv", index = False)

크롤링 완료


In [10]:
product_detail = []
pro_num = 0
num = 0

for i in tqdm_notebook(range(len(df))):
    num+=1
    while pro_num < num:
        if pro_num == 361:
            break
        try:
            number = df["link"][i].split("/")[4]
            url = f"https://kream.co.kr/products/{number}"
            time.sleep(random.randint(1, 3))
            driver.get(url)

            product_items = driver.find_element_by_class_name('detail_product')
            items = product_items.text.split("\n")

            product_dict = {
                "link" : url,
                "model" : items[1],
                "release" : items[3],
                "color" : items[5],
                "price" : items[7]
            }
            product_detail.append(product_dict)
            pro_num += 1
            print(f"{i+1}번 크롤링 완료")
        except:
            driver.refresh()

df1 = pd.DataFrame(product_detail)

<ipython-input-10-5a53ca7c28d8>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(df))):


  0%|          | 0/360 [00:00<?, ?it/s]

1번 크롤링 완료
2번 크롤링 완료
3번 크롤링 완료
4번 크롤링 완료
5번 크롤링 완료
6번 크롤링 완료
7번 크롤링 완료
8번 크롤링 완료
9번 크롤링 완료
10번 크롤링 완료
11번 크롤링 완료
12번 크롤링 완료
13번 크롤링 완료
14번 크롤링 완료
15번 크롤링 완료
16번 크롤링 완료
17번 크롤링 완료
18번 크롤링 완료
19번 크롤링 완료
20번 크롤링 완료
21번 크롤링 완료
22번 크롤링 완료
23번 크롤링 완료
24번 크롤링 완료
25번 크롤링 완료
26번 크롤링 완료
27번 크롤링 완료
28번 크롤링 완료
29번 크롤링 완료
30번 크롤링 완료
31번 크롤링 완료
32번 크롤링 완료
33번 크롤링 완료
34번 크롤링 완료
35번 크롤링 완료
36번 크롤링 완료
37번 크롤링 완료
38번 크롤링 완료
39번 크롤링 완료
40번 크롤링 완료
41번 크롤링 완료
42번 크롤링 완료
43번 크롤링 완료
44번 크롤링 완료
45번 크롤링 완료
46번 크롤링 완료
47번 크롤링 완료
48번 크롤링 완료
49번 크롤링 완료
50번 크롤링 완료
51번 크롤링 완료
52번 크롤링 완료
53번 크롤링 완료
54번 크롤링 완료
55번 크롤링 완료
56번 크롤링 완료
57번 크롤링 완료
58번 크롤링 완료
59번 크롤링 완료
60번 크롤링 완료
61번 크롤링 완료
62번 크롤링 완료
63번 크롤링 완료
64번 크롤링 완료
65번 크롤링 완료
66번 크롤링 완료
67번 크롤링 완료
68번 크롤링 완료
69번 크롤링 완료
70번 크롤링 완료
71번 크롤링 완료
72번 크롤링 완료
73번 크롤링 완료
74번 크롤링 완료
75번 크롤링 완료
76번 크롤링 완료
77번 크롤링 완료
78번 크롤링 완료
79번 크롤링 완료
80번 크롤링 완료
81번 크롤링 완료
82번 크롤링 완료
83번 크롤링 완료
84번 크롤링 완료
85번 크롤링 완료
86번 크롤링 완료
87번 크롤링 완료
88번 크롤링 완료
89번 크롤링 완료
90번 크롤링 완료
91번 크롤링 완료
92번 크롤링 

# merge 및 전처리

In [15]:
def preprocessing(date):
    # 최종 merge
    total = pd.merge(df, df1, on="link")
    # 상품번호 분리
    total["product_num"] = total["link"].apply(lambda x: x.split("/")[4])
    # 컬럼 재정렬
    total = total[["brand", "en_name", "kor_name", "product_num", "model", "release", "color", 
                   "price", "re_price", "wish", "review", "link"]]
    # 저장
    total.to_csv(f"{date}_kream.csv", index = False)
    return total

In [16]:
preprocessing(220217)

,brand,en_name,kor_name,product_num,model,release,color,price,re_price,wish,review,link
0,Jordan,Jordan 1 Retro High OG Black Mocha,조던 1 레트로 하이 OG 블랙 모카,25922,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원","690,000",2.3만,"5,182",https://kream.co.kr/products/25922
1,Nike,Nike Dunk Low Retro Black,나이키 덩크 로우 레트로 블랙,28029,DD1391-100,21/01/14,WHITE/BLACK,"129,000원","264,000",6.5만,"8,781",https://kream.co.kr/products/28029
2,Jordan,Jordan 1 Mid Light Smoke Grey Anthracite,조던 1 미드 라이트 스모크 그레이 앤트러사이트,47257,554724-078,22/01/24,LIGHT SMOKE GREY/WHITE-ANTHRACITE,"€120 (약 163,200원)","338,000",1.1만,114,https://kream.co.kr/products/47257
3,Nike,Nike Dunk Low Retro Championship Court Purple,나이키 덩크 로우 레트로 챔피언쉽 코트 퍼플,47887,DD1391-104,22/02/11,WHITE/COURT PURPLE,"129,000원","239,000",1.1만,364,https://kream.co.kr/products/47887
4,Nike,(W) Nike Dunk Low Black,(W) 나이키 덩크 로우 블랙,28030,DD1503-101,21/01/04,WHITE/BLACK,"129,000원","278,000",4.5만,"3,344",https://kream.co.kr/products/28030
...,...,...,...,...,...,...,...,...,...,...,...,...
355,Supreme,Supreme Box Logo Hooded Sweatshirt White - 21FW,슈프림 박스로고 후드 스웨트셔츠 화이트 - 21FW,46830,-,21/12/09,WHITE,"$168 (약 200,900원)","530,000",317,8,https://kream.co.kr/products/46830
356,Mihara Yasuhiro,Maison Mihara Yasuhiro Peterson OG Sole Canvas...,메종 미하라 야스히로 피터슨 OG 솔 캔버스 로우 스니커 블랙,25707,A01FW702-BLACK,-,BLACK,"¥30,800 (약 318,500원)","650,000","3,009",132,https://kream.co.kr/products/25707
357,Arc'teryx,Arc'teryx Beta Jacket Black - 22SS,아크테릭스 베타 자켓 블랙 - 22SS,50099,29090,-,BLACK,"650,000원","505,000",347,5,https://kream.co.kr/products/50099
358,Chanel,Chanel Clutch With Chain Lambskin & Gold Black,샤넬 체인 미니백 램스킨 & 골드 메탈 블랙,49241,A81633-Y04059-94305,-,GOLD/BLACK,-,"3,648,000",249,1,https://kream.co.kr/products/49241
